### Intall libraries

In [50]:
from IPython.display import clear_output

!pip install nltk
clear_output(wait=False)

### Spelling correction

inspired by Norvig's spelling correction algorithm

0. Load n-gram model

In [51]:
import pickle

with open(r"models/trigram_model_shakespeare.pkl", "rb") as f:
    ngram_model = pickle.load(f)

print("model loaded")

model loaded


In [52]:
print(ngram_model.get(("the", "great"), {}))

{'toe': 9.835997680775283e-05, 'aufidius': 8.80062950385157e-05, 'king': 8.80062950385157e-05, 'rich': 8.80062950385157e-05, 'chamber': 8.80062950385157e-05, 'lord': 8.80062950385157e-05, 'commanding': 8.80062950385157e-05, 'apollo': 9.318313592313427e-05, 'comfort': 8.80062950385157e-05, 'pompey': 8.80062950385157e-05, 'soldier': 8.80062950385157e-05, 'traveller': 8.80062950385157e-05, 'desire': 8.80062950385157e-05}


In [53]:
vocab = set(word for counts in ngram_model.values() for word in counts)
vocab_size = len(set(word for counts in ngram_model.values() for word in counts))
print(vocab_size)

12072


1. Generate possible candidates

In [54]:
def edits1(word):
    letters = "abcdefghijklmnopqrstuvwxyz"
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]

    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1))

In [55]:
edits1("cat")

{'aat',
 'acat',
 'act',
 'at',
 'bat',
 'bcat',
 'ca',
 'caa',
 'caat',
 'cab',
 'cabt',
 'cac',
 'cact',
 'cad',
 'cadt',
 'cae',
 'caet',
 'caf',
 'caft',
 'cag',
 'cagt',
 'cah',
 'caht',
 'cai',
 'cait',
 'caj',
 'cajt',
 'cak',
 'cakt',
 'cal',
 'calt',
 'cam',
 'camt',
 'can',
 'cant',
 'cao',
 'caot',
 'cap',
 'capt',
 'caq',
 'caqt',
 'car',
 'cart',
 'cas',
 'cast',
 'cat',
 'cata',
 'catb',
 'catc',
 'catd',
 'cate',
 'catf',
 'catg',
 'cath',
 'cati',
 'catj',
 'catk',
 'catl',
 'catm',
 'catn',
 'cato',
 'catp',
 'catq',
 'catr',
 'cats',
 'catt',
 'catu',
 'catv',
 'catw',
 'catx',
 'caty',
 'catz',
 'cau',
 'caut',
 'cav',
 'cavt',
 'caw',
 'cawt',
 'cax',
 'caxt',
 'cay',
 'cayt',
 'caz',
 'cazt',
 'cbat',
 'cbt',
 'ccat',
 'cct',
 'cdat',
 'cdt',
 'ceat',
 'cet',
 'cfat',
 'cft',
 'cgat',
 'cgt',
 'chat',
 'cht',
 'ciat',
 'cit',
 'cjat',
 'cjt',
 'ckat',
 'ckt',
 'clat',
 'clt',
 'cmat',
 'cmt',
 'cnat',
 'cnt',
 'coat',
 'cot',
 'cpat',
 'cpt',
 'cqat',
 'cqt',
 'cra

In [56]:
len(edits1("cat"))

182

In [57]:
len(edits2("cat"))

14352

In [58]:
def generate_candidates(word, vocab, max_distance=2):
    candidates = {word} if word in vocab else set()
    valid_edits1 = edits1(word) & vocab
    if valid_edits1:
        return valid_edits1

    if max_distance > 1:
        word_edits2 = edits2(word)
        return word_edits2 & vocab
    return candidates

In [59]:
generate_candidates("caf", vocab)

{'calf', 'can', 'cap', 'car', 'cat'}

In [60]:
EPS = 1e-12

def get_ngram_probability(context, word):
    """Compute P(word | context) using the trained N-gram model."""
    ngram = tuple(context) #+ (word,)
    if ngram not in ngram_model:
        return EPS
    if word not in ngram_model[ngram]:
        return EPS
    return ngram_model.get(ngram, EPS)[word]

In [61]:
get_ngram_probability(("the", "great"), "shakespeare")

1e-12

In [62]:
get_ngram_probability(("the", "great"), "king")

8.80062950385157e-05

In [71]:
import nltk

n=3

def correct_sentence(sentence, vocab):
    words = nltk.word_tokenize(sentence.lower())
    corrected_words = []

    for i, word in enumerate(words):
        if word in vocab:
            corrected_words.append(word)
            continue

        context = tuple(corrected_words[-(n-1):]) if i >= n-1 else tuple(corrected_words)
        candidates = generate_candidates(word, vocab)

        best_candidate = max(candidates, key=lambda w: get_ngram_probability(context, w))
        corrected_words.append(best_candidate)
    return ' '.join(corrected_words)

In [72]:
correct_sentence("to be or not to ve", vocab)

'to be or not to be'